In [26]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors 
import time
list_csv = ['data/attraction_places.csv',
           'data/hotel_places.csv',
           'data/restaurant_places.csv']
attraction_e2 =  KeyedVectors.load('model/attraction_tag_e2.model')
attraction_e3 =  KeyedVectors.load('model/attraction_tag_e3.model')
attraction_e4 =  KeyedVectors.load('model/attraction_tag_e4.model')
attraction_e5 =  KeyedVectors.load('model/attraction_tag_e5.model')
attraction_user =  KeyedVectors.load('model/attraction_user.model')
#model2 =  Word2Vec().wv.load('model/eng_hotel_tag.model')
#model3 =  Word2Vec().wv.load('model/restaurant_tag.model')

In [2]:
entities_list = ['324890', '3808315', '590748']

In [3]:
def rank(srcwv, entry, positive=None, negative=None):
    subwv = srcwv[entry]
    newwv = Word2Vec(size=len(subwv[0]))
    # add entries to new vector
    for i in range(len(entry)):
        newwv.wv.add(entry[i], subwv[i])
    # add p, n to new vector
    for w in positive:
        newwv.wv.add(w, srcwv[w])
    for w in negative:
        newwv.wv.add(w, srcwv[w])
    rank = newwv.most_similar(positive=positive,
                              negative=negative,
                              topn=len(entry))
    return rank

In [4]:
rank(attraction_e5, entities_list, positive=['320359'], negative=['10020510'])

/home/jahn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/home/jahn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('324890', 0.3276669383049011),
 ('3808315', 0.3103531301021576),
 ('590748', 0.2649676203727722)]

In [27]:
df_place

,class,placeId,name_kor,name_eng,location
0,attractions,10020510,한성백제문화제,Hanseong Baekje Cultural Festival,"37.520847,127.11603"
1,attractions,10020511,하이서울페스티벌,Hi Seoul Festival,"서울 (Multiple locations, including Seoul Plaza ..."
2,attractions,10020529,서울불꽃축제,Seoul International Fireworks Festival,"37.528095,126.92417"
3,attractions,10020530,서울빛초롱축제,Seoul Lantern Festival,"37.569157,127.00403"
4,attractions,10130671,학아재미술관,Hagajae Museum,"37.577007,126.9904"
5,attractions,10183192,석촌호수,Seokchon Lake,"37.508343,127.1"
6,attractions,10210051,누죤,Nuzzon,서울 중구 을지로45길 62
7,attractions,10253198,서울파이낸스센터,Seoul Finance Center,서울 중구 세종대로 136
8,attractions,10270564,코엑스,COEX Center,서울 강남구 영동대로 513
9,attractions,10288072,디뮤지엄,D Museum,"37.53687,127.01137"


In [30]:
model = attraction_e5
df_place = pd.read_csv(list_csv[0])
p = ['10270564']
n = []
print('positive: ', [df_place[df_place['placeId']==int(i)].name_kor.tolist()[0]
                     for i in p])
print('negative: ', [df_place[df_place['placeId']==int(i)].name_kor.tolist()[0]
                     for i in n])
print('상위 10개 유사도')
top10 = model.most_similar(positive=p, negative=n, topn=569)
cnt = 0
result = pd.DataFrame(columns=['name', 'similarity'])
for place in top10:
    name = df_place[df_place['placeId'] == int(place[0])].name_kor.tolist()[0]
    if pd.isna(name):
        name = df_place[df_place['placeId'] == int(place[0])].name_eng.tolist()[0]
    result.loc[cnt] = [name,place[1]]
    cnt+=1
result

positive:  ['코엑스']
negative:  []
상위 10개 유사도


/home/jahn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,name,similarity
0,아이파크몰,0.507827
1,IFC 몰,0.466631
2,신세계 강남점,0.459212
3,,0.445607
4,코엑스 센터,0.444016
5,디큐브 시티,0.443519
6,이태원 앤틱 가구 거리,0.434878
7,갤러리아 백화점 본점,0.430926
8,신세계 백화점 본점,0.430766
9,신촌,0.428955


In [11]:
#유사도 행렬 추출
index = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(model.syn0))

/home/jahn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.
/home/jahn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
